In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
import pandas_datareader as dr
import yfinance as yf
import numpy as np
import glob
from collections import Counter
from ast import literal_eval


In [2]:
os.chdir('..')
os.chdir('..')
cwd = os.getcwd()
print(cwd)

D:\Programming\Subreddit-Stock-Prediction


In [3]:
path = r'data' # use your path
all_files = glob.glob(path + "/*.jsonl")

tickCnt2 = Counter()

for filename in all_files:
    print(filename)
    df = pd.read_json(filename,lines=True)
    for x in df['tickers']:
        if len(x) > 0:
            y, z = x[0]
            tickCnt2[y] += 1

data\daytrading_comments.jsonl
data\daytrading_submission.jsonl
data\dividends_comments.jsonl
data\dividends_submission.jsonl
data\economy_comments.jsonl
data\economy_submission.jsonl
data\investing_comments.jsonl
data\investing_submission.jsonl
data\options_comments.jsonl
data\options_submission.jsonl
data\securityanalysis_comments.jsonl
data\securityanalysis_submission.jsonl
data\stockmarket_comments.jsonl
data\stockmarket_submission.jsonl
data\stocks_comments.jsonl
data\stocks_submission.jsonl


In [21]:
top_n = 30
top_list = tickCnt2.most_common(top_n)
print(top_list[-10:])

[('LIVE', 537), ('BA', 525), ('GM', 510), ('B', 485), ('MU', 477), ('F', 470), ('V', 453), ('IBM', 451), ('TWTR', 433), ('TV', 419)]


In [22]:
blacklist = ["TD", "AI", "X", "RSI", "LIVE", "B", "TV"]
top_list = list(filter(lambda ticker_and_count: ticker_and_count[0] not in blacklist, top_list))
ticker_list = list(map(lambda t_a_c: t_a_c[0], top_list))

In [23]:
start = datetime(2014,1,1)
end = datetime(2021,1,1)

stock_data = {}
for ticker in ticker_list:
    stock = dr.data.DataReader(ticker, data_source='yahoo', start=start, end=end)
    stock_data[ticker] = stock

In [24]:
submission_sentiments = pd.read_csv("data/LabelledData/submissions_with_tickers_labelled.csv")
submission_sentiments.set_index("name", inplace=True)
comment_sentiments = pd.read_csv("data/LabelledData/comments_with_tickers_labelled.csv")

# Get Top
comment_sentiments["tickers"] = comment_sentiments["tickers"].apply(lambda x: literal_eval(str(x)))
comment_sentiments["ticker"] = comment_sentiments["tickers"].map(lambda tickers: tickers[0][0] if len(tickers) > 0 else np.NaN)
# print(ticker_list)
comment_sentiments = comment_sentiments[comment_sentiments["ticker"].isin(ticker_list)]
comment_sentiments.dropna()
# print(comment_sentiments["ticker"])

,Unnamed: 0,submission_name,id,text,score,created_utc,tickers,neg,neu,pos,ticker
0,0,t3_8dosib,dxot506,Im thinking aobc or else ford. both have rough...,3.0,1.524242e+09,"[[F, 1]]",0.939834,0.045941,0.014225,F
1,1,t3_a4et2x,ebe85cf,Does anyone have a excel sample sheet they can...,3.0,1.544318e+09,"[[GOOG, 1]]",0.054193,0.913546,0.032261,GOOG
6,6,t3_an4n5g,efqslh9,Define cheap. For me I'm putting a bunch of mo...,3.0,1.549309e+09,"[[MSFT, 1]]",0.016944,0.317534,0.665522,MSFT
8,8,t3_an4n5g,efrrain,T,3.0,1.549335e+09,"[[T, 1]]",0.057315,0.883271,0.059414,T
19,19,t3_bdmdvj,elfucsc,I am thinking about doing this strategy as wel...,2.0,1.555871e+09,"[[FB, 1]]",0.023970,0.929311,0.046719,FB
...,...,...,...,...,...,...,...,...,...,...,...
41597,41597,t3_knnc52,ghm73op,Azure will provide consistent growth for the n...,2.0,1.609431e+09,"[[MSFT, 2], [AMZN, 1]]",0.011626,0.846059,0.142315,MSFT
41598,41598,t3_knnc52,ghndxub,Ask yourself one question \n\nIs Microsoft off...,2.0,1.609454e+09,"[[MSFT, 1]]",0.014535,0.902674,0.082791,MSFT
41599,41599,t3_knnc52,ghlw67q,Tldr: MSFT to the moon 🚀🚀🚀🚀🚀🚀🚀,2.0,1.609425e+09,"[[MSFT, 1]]",0.018985,0.918247,0.062768,MSFT
41600,41600,t3_knjs1t,ghkwxe8,It’s not a battery day. It’s just NIO day. The...,6.0,1.609393e+09,"[[NIO, 1]]",0.057290,0.912919,0.029791,NIO


In [25]:
avg_t_span = timedelta(days=1)
span_interval = timedelta(days=1)
future_t_interval = timedelta(days=7)

eps = .02

for filename in all_files:
    if not filename.endswith("submission.jsonl"):
        continue

    subreddit_path = filename.replace("_submission.jsonl", "")
    print(filename)
    
    df = pd.read_json(filename,lines=True)
    df["date"] = pd.to_datetime(df['created_utc'],unit='s')
    
    # clean dataset
    df["ticker"] = df["tickers"].map(lambda tickers: tickers[0][0] if len(tickers) > 0 else np.NaN)
    df = df[df["ticker"].isin(ticker_list)]
    df.dropna()
    df.set_index("date", inplace=True)
    df.sort_index(inplace=True)
    
    features = []
    
    t = start
    while t <= end - avg_t_span:
#         mask = (df >= t) & (df <= t + avg_t_span)
#         submissions = df.loc[mask]
#         print(df.index)
#         print(t)
#         if(t.year != (t + avg_t_span).year):
#             print(t)

        grouped_submissions = df[str(t):str(t+avg_t_span)]
        if len(grouped_submissions) == 0:
            t += span_interval
            continue
        
        comment_mask = comment_sentiments["submission_name"].isin(grouped_submissions["name"])
        comment_mask = comment_mask & comment_sentiments["ticker"].isin(ticker_list)
        grouped_comments = comment_sentiments.loc[comment_mask]
        
        for ticker in ticker_list:
            submissions = grouped_submissions[grouped_submissions["ticker"] == ticker]
            if len(submissions) == 0:
                continue
            
            comment_mask = grouped_comments["submission_name"].isin(submissions["name"])
            comment_mask = comment_mask & (grouped_comments["ticker"] == ticker)
            comments = grouped_comments.loc[comment_mask]
            
            s_scores = submissions["score"].values
            s_uv_ratio = submissions["upvote_ratio"].values

            s_sentiments = submission_sentiments.loc[submissions["name"],["neg","neu","pos"]].values
            s_sentiments *= (s_scores * s_uv_ratio).reshape(-1,1)
            s_sentiments = s_sentiments.sum(axis=0)

            if len(comments) == 0:
                c_sentiments = np.zeros(3)
            else:
                c_sentiments = comments[["neg","neu","pos"]].values
                c_scores = comments["score"].values
                c_sentiments = (c_sentiments * c_scores.reshape(-1,1)).sum(axis=0)
                
            # Get Stock Stuff
            stock = stock_data[ticker]
            cur_stock_idx = stock.index.get_loc(t, method="nearest")
            future_stock_idx = stock.index.get_loc(t + avg_t_span, method="nearest")
            cur_stock = stock.iloc[cur_stock_idx]["Open"] #Note : is rounded to nearest date
            future_stock = stock.iloc[future_stock_idx]["Close"]
            
            percent = (future_stock - cur_stock) / cur_stock
            if np.isnan(percent):
                continue
            
            if np.abs(percent) <= eps:
                percent = 0
            
            label = np.sign(percent)
            
            feature_vector = np.concatenate((s_sentiments, c_sentiments, np.array([label]), np.array([percent])), axis=0)
            features.append(feature_vector)
        
        t += span_interval
    pd.DataFrame(np.array(features)).to_csv(f"{subreddit_path}_features_labels.csv", index=False)
            


data\daytrading_submission.jsonl
data\dividends_submission.jsonl
data\economy_submission.jsonl
data\investing_submission.jsonl
data\options_submission.jsonl
data\securityanalysis_submission.jsonl
data\stockmarket_submission.jsonl
data\stocks_submission.jsonl
